In [8]:
## Importar librerias
from linares_plot import *
import statsmodels.api as sm
from scipy import stats
import statsmodels.formula.api as smf
from statsmodels.genmod.bayes_mixed_glm import BinomialBayesMixedGLM
import itertools
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels
import scikit_posthocs as sp
from statsmodels.stats.proportion import proportions_ztest


%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
### Open data
### Concatenate the different shets into one dataframe

xls = pd.ExcelFile('C:\\Users\\David\\Documents\\GitHub\\episodic-memory-magic\\4meses.xlsx') ##clean
sheets_int = xls.sheet_names[-3:] #Now you can list all sheets in the file ##when doing it for real, [-3:]
frames=[]
for sheet in sheets_int:
    frames.append(pd.read_excel('C:\\Users\\David\\Documents\\GitHub\\episodic-memory-magic\\4meses.xlsx', sheet_name=sheet ))

                  
df_data = pd.concat(frames, ignore_index=True)
print(len(df_data) )## all shets concatenated

118


C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [21]:
## funciones

def map_book(tokens): #function to go from list of words to dictionari (take off punctuation and repetitions
    words_wp = []
    hash_map = {}
    if tokens is not None:
        for element in tokens:
            # Remove Punctuation
            word = element.replace(",","")
            word = word.replace(".","")
            word = word.replace("-","")
            word = word.replace(")","")
            word = word.replace("(","")
            words_wp.append(word)

            # Word Exist?
            if word in hash_map:
                hash_map[word] = hash_map[word] + 1
            else:
                hash_map[word] = 1

        return hash_map, words_wp
    else:
        return None


def give_numbers(wordlist): # function to remove stop words
    numbers=['1', '2', '22', '23', '3', '4', '5', '6', '7', '8', '9', '0'] 
    return [w for w in wordlist if w in numbers]


def effects_detected(text):
    words_all=[]
    [words_all.append(  text.split()[n]  ) for n in range( len(text.split()))]
    dict_words, list_words = map_book(words_all) #take off the puntuation and take off repetitions
    sortedkeys=sorted(dict_words.keys(), key=lambda x:x.lower()) ##list of uniques in alphabetical order 
    efectos = give_numbers(sortedkeys) # here I do not mind if 5 is reported 3 times, just yes or no.
    return efectos, list_words



def separate_sentences_by_effect(text):
    ## effects detected
    efectos, list_words = effects_detected(text)
    #start of sentences
    start_sentences=[]
    for e in efectos:
        arr = np.where(np.array(list_words)==e)[0] ## here I care if 2 is reported 3 times. I want all the sentences
        for i in range(len(arr)):
            start_sentences.append(arr[i])

    start_sentences.sort()
    start_sentences.insert(len(start_sentences), len(list_words)) ## otherwise you miss the last resport
    #get the whole sentence by index
    complete_sentences=[]
    for ss in range(len(start_sentences)-1):
        sentence =  list_words[start_sentences[ss]: start_sentences[ss+1]  ]
        sentence = " ".join(sentence)
        complete_sentences.append( sentence)
    
    ## by definition, each sentence start with a number
    dict_by_effect={}
    for sentence in complete_sentences:
        eff_sent = sentence.split()[0]
        if eff_sent not in list(dict_by_effect.keys()):
            dict_by_effect[eff_sent]= [ " ".join(sentence.split()[1:]) ]
        else:
            dict_by_effect[eff_sent].append( " ".join(sentence.split()[1:])  )
    
    ##   
    return complete_sentences, dict_by_effect




In [22]:
## saliencias (palabras que se buscan para determinar saliencias)

saliencias_1= [['comodín', 'comodin', 'comodí', 'comodi', 'joker',
                 'transposición', 'transposition', 'transposició'],
               ['estuche', 'estoig', 'buida', 'caja', 'caixa', 'estuche', 'estoig', 'predicció', 'prediccio',
                'predicción', 'prediccion', 'case', 'prediction', 'box'],
               ['montones',  'montón', 'monton', 'pilas', 'pilons', 'divide', 'divideix', 'mitades', 'mitad', 'meitat', 'media',
                'partir', 'dividir', 'piles', 'division', 'half', 'bifurcation', 'realidades', 'realitats', 'pierdes', 'perds',
                'realities', 'lose', 'lost', 'miss', 'perder', 'perdiste', 'perdre', 'parte', 'partir', 'dividir', 'cortar',
               'bifurcación', 'bifurcació', 'bifurcacions', 'bifurcaciones', 'tiempo', 'temps', 'time', 'vez', 'vegada',
                'cortes', 'reductores', 'reducir', 'corte', 'izquierda', 'derecha', 'dividiendo', 'separando', 'mini',
               'dividint', 'bifuracción', 'cortando', 'grupos', 'pequeños', 'numero', 'número', 'nombre', 'decreixent',
               'izquierdo', 'derecho', 'escapçava', 'mazo'],
               ['color', 'blau', 'vermell', 'azul', 'rojo', 'colores', 'dorso', 
                'detrás', 'detras', 'darrera', 'red', 'blue', 'back', 'blaus',
                'backs', 'behind', 'blaves', 'vermelles', 'rojas', 'azules', 'vermella', 'blava'],
               ['encore', 'transformación', 'transformació', 'transformation', 'transforman'],
              ['as','ace', 'tres', 'three', 'cuatro', 'quatre', 'four', 'cinc', 'cinco', 'five', 'seis', 
              'sis', 'six', 'siete', 'set', 'seven', 'ocho', 'vuit', 'eight', 'nueve', 'nou', 'nine', 'diez',
              'deu', 'ten', 'jota', 'jotas', 'jack', 'dama', 'damas', 'reina', 'reinas', 'queen', 'rey', 'rei', 'reis', 'reyes', 
               'king', 'picas', 'piques', 'spades', 'treboles', 'tréboles', 'trebol', 'trèbol', 'trèbols', 'trebols', 'culbs',
              'diamantes', 'rombos', 'diamants', 'rombes', 'diamonts', 'diamonds', 'corazones', 'corazón', 'cors', 'cor', 'heard',
              'heards'],
              ['chico', 'noi', 'chica', 'noia', 'raquel']]


saliencias_2 = [['cambian', 'frotar', 'avall', 'cambio', 'taula', 'mesa', 'table', 'tapete', 'tapet', 'rug', 'transformar', 'transformació',
                 'fregar', 'rub', 'cambiaba', 'canviava', 'cambia', 'canvia', 'canvien', 'cambiar', 'convierte', 'convierten',
                 'change', 'changed', 'Change', 'valor', 'value', 'color', 'fregava', 'frotaba', 'frotó', 'rubbed'], 
                ['aparecían', 'appear', 'aparecer', 'aparecen','salían', 'salian', 'sortien', 'appeared', 'moneda',
                 'monedas', 'monedes' , 'coins',  'euros', 'dinero', 'coin', 'money', 'diners', 
                 'entre', 'between', 'dinerito', 'moendas', 'pequeñas', 'pequeña', 'little'],
                ['as','ace', 'tres', 'three', 'cuatro', 'quatre', 'four', 'cinc', 'cinco', 'five', 'seis', 
              'sis', 'six', 'siete', 'set', 'seven', 'ocho', 'vuit', 'eight', 'nueve', 'nou', 'nine', 'diez',
              'deu', 'ten', 'jota', 'jotas', 'jack', 'dama', 'damas', 'reina', 'reinas', 'queen', 'rey', 'rei', 'reis', 'reyes', 
               'king', 'picas', 'piques', 'spades', 'treboles', 'tréboles', 'trebol', 'trèbol', 'trèbols', 'trebols', 'culbs',
              'diamantes', 'rombos', 'diamants', 'rombes', 'diamonts', 'diamonds', 'corazones', 'corazón', 'cors', 'cor', 'heart',
              'hearts'],
               ['chico', 'noi', 'chica', 'raquel', 'sortir', 'noia', 'salir', 'baixar', 'escenari', 'escenario', 'tarima', 'front'],
               ['oportunidades', 'oportunidad', 'oportunitats', 'oportunitat', 'oportunity', 'intentos', 'intents'],
               ['desaparecía', 'desaparecer', 'disappear', 'desvanecía', 'desapareixía', 'esfumaba', 'desparecia'],
               ['oreja', 'orella', 'orejas', 'orelles', 'ear', 'pelo', 'cabell', 'pelo']]


saliencias_3 = [['monedes', 'amagades', 'sota', 'debajo', 'tapades', 'tapadas', 'under', 'hide',
                'escondidas', 'cards', 'coins', 'cartas', 'monedas', 'tapades', 'bajo'],
               ['viajaban', 'matrix', 'travell', 'movían', 'reunían', 'juntaban', 'separaban', 'Matrix',
               'viatjaven', 'viajan', 'movien', 'moviendo','vaitjes', 'movian', 'move', 'moved', 'travelled',
               'cambian', 'cambiaban', 'cambiaba','changed', 'change', 'canvia', 'canviava', 'mouen',
               'posición', 'position', 'places', 'posició', 'posiciones', 'posicions', 'mateixa', 'pasaban']]


saliencias_4 = [['alma', 'invisible', 'soul', 'ànima', 'invisible', 'fake', 'mentira', 'mentida', 'imaginar', 'imaginario',
                 'imaginaria', 'imaginary', 'anima', 'esencia', 'essència', 'essencia', 'invisibles', 'invisibles', 'invisible',
                 'imaginàries', 'imaginària', 'imaginarias', 'falsa', 'simuli', 'simulada', 'simular', 'lanzaba', 
                 'imaginariament', 'imaginariamente', 'vola', 'esperit', 'simulant', 'veure', 'simbólica', 'pasaba',
                'mímica', 'esperit', 'inexistent'],
               ['girada', 'revered', 'reves', 'revés', 'side', 'vuelta', 'turned', 'medio', 'middle', 'mig', 'prediction',
               'predicción', 'predicció', 'volta', 'capgirada'],
                ['as','ace', 'dos', 'two', 'tres', 'three', 'cuatro', 'quatre', 'four', 'cinc', 'cinco', 'five', 'seis', 
              'sis', 'six', 'siete', 'set', 'seven', 'ocho', 'vuit', 'eight', 'nueve', 'nou', 'nine', 'diez',
              'deu', 'ten', 'jota', 'jotas', 'jack', 'dama', 'damas', 'reina', 'reinas', 'queen', 'rey', 'rei', 'reis', 'reyes', 
               'king', 'picas', 'piques', 'spades', 'treboles', 'tréboles', 'trebol', 'trèbol', 'trèbols', 'trebols', 'culbs',
              'diamantes', 'rombos', 'diamants', 'rombes', 'diamonts', 'diamonds', 'corazones', 'corazón', 'cors', 'cor', 'heart',
              'hearts'],
               ['chico', 'chica', 'raquel']]


saliencias_5 = [['transforman', 'transformen', 'transform', 'cambiaban', 'canviaven', 'changed', 'material', 'old', 'viajes',
                'cobre', 'plata', 'copper', 'bronce', 'antiguas', 'silver', 'noves', 'nuevas', 'antigues', 'noves', 'color', 
                 'colores', 'colors', 'gris', 'marrón', 'plateadas', 'platejades', 'coure', 'platejat', 'manterial'],
                ['vaso', 'baso', 'glass', 'taza', 'got', 'tassa', 'mug', 'vas', 'vasos', 'gots', 'cup', 'cups', 'glasses', 'mugs'],
                ['mesa', 'table', 'taula', 'mantel', 'trespassaven'],
                ['cuerpo', 'mano', 'hand', 'ma', 'mà', 'hands', 'manos', 'mans', 'cos', 'body', 'brazo', 'braç', 'brazos', 'braços'],
                ['salir', 'sortir', 'salían', 'sortien', 'quit', 'leave', 'left', 'salia', 'salía', 'sortia', 'revés', 'reves']]


saliencias_6 = [['deletreos', 'deletrear', 'deletraba', 'deletreba', 'deletreaba', 'deletreo', 'deltreó', 'deletreando', 'número', 'numero', 'number',
                 'letra', 'letras', 'palabra', 'palabras', 'word', 'words', 'lletra', 'paraula', 'paraules', 'mot', 'mots',
                 'spell', 'spelling', 'spelled', 'delletrejar', 'delletrejaba', 'delletreja', 'deletrejant', 'lletrejar',
                'contar', 'contaba', 'contando', 'comptar', 'comptava', 'comptant', 'comptaven', 'escribiendo', 'escrivint',
                'nom', 'nombre', 'deletreadas', 'lletreig', 'deletrea', 'deletrejava', 'delletrejat', 'nominar', 'anomenar',
                'cridar', 'deletreandolas', 'deletrreando', 'deletrearla', 'deletrean', 'deletreabas', 'delerejades', 'deletreaba',
                'deletreándolas', 'deletrar', 'lletres', 'deletrejar', 'nombra', 'contaban', 'nombrarlas', 'llamando',
                'contant'],
               ['tres', 'three', 'espectadores', 'spectators', 'people', 'persones', 'espectadors', 'públic', 'public', 'actuantes', 
                'público', 'publico', 'auditorio', 'auditori'],
                ['española', 'spanish', 'espanyola', 'antiga', 'antigua','bastos', 'oros', 'copas', 'espadas'],
                ['cualquiera', 'any', 'qualsevol', 'atzar', 'azar', 'random'],
                ['fallado', 'salido', 'equivocó', 'falló', 'fallido', 'errores', 'mistakes', 'work', 'mistake', 'error', 'be', 'bien',
                'anulado', 'bé', 'sortir', 'fallit', 'mal', 'falla'],
               ['chico', 'chica', 'noi', 'noia', 'raquel']]


saliencias_7 = [['blanques', 'blanca', 'blancas', 'white', 'blanquear', 'blanco', 'blanc', 'blancos'],
               ['Granada', 'granada', 'amigo', 'amic', 'friend', 'andalusa', 'andalús'],
               ['tiempo', 'time', 'dimensions', 'dimensiones', 'temps', 'pasado', 'passat', 'past',
                'presente', 'present', 'futuro, futur', 'future', 'somnis', 'sueños', 'onírica', 'dreams',
               'poema', 'poesía', 'poètico', 'poem', 'poesia', 'historieta'],
               ['reunien', 'ruenion', 'deapararecían', 'desparecen', 'reunen', 'reyes', 'ases', 'disappear',
               'convertían', 'travell', 'viajan', 'viatgen', 'viatjaven', 'viajaban', 'reunían', 'reunión',
               'montón', 'pile', 'montones', 'pila', 'vanish', 'vanished', 'appear', 'aparecían', 'apareixien',
               'apareixen', 'aparecen', 'assamblea', 'assembly', 'asamblea', 'aparecido', 'aparescut'],
                ['as','ace', 'dos', 'two', 'tres', 'three', 'cuatro', 'quatre', 'four', 'cinc', 'cinco', 'five', 'seis', 
              'sis', 'six', 'siete', 'set', 'seven', 'ocho', 'vuit', 'eight', 'nueve', 'nou', 'nine', 'diez',
              'deu', 'ten', 'jota', 'jotas', 'jack', 'dama', 'damas', 'reina', 'reinas', 'queen', 'rey', 'rei', 'reis', 'reyes', 
               'king', 'picas', 'piques', 'spades', 'treboles', 'tréboles', 'trebol', 'trèbol', 'trèbols', 'trebols', 'culbs',
              'diamantes', 'rombos', 'diamants', 'rombes', 'diamonts', 'diamonds', 'corazones', 'corazón', 'cors', 'cor', 'heart',
              'hearts']]



saliencias_8 = [['instrucciones', 'instructions', 'instruccions', 'escrita', 'written', 'notas', 'notes', 'pistas', 'pista',
                'pistas', 'clues', 'detective', 'detective', 'tahur', 'taur', 'tahúr', 'taúr', 'trampas', 'tramposo',
                'trampós', 'cardshark', 'expresions', 'acciones', 'misatges', 'missatge', 'missatges', 'mensaje', 'mensajes', 'messages',
                'frases', 'escrites', 'escritas', 'órdenes', 'ordres', 'indicacions', 'indicaciones', 'tinta', 'ink',
                'marca', 'marcada', 'decía', 'manda', 'letras', 'encuentra', 'historia', 'història', 'diciendo',
                'pasos', 'steps', 'seguir', 'habla', 'información', 'informacion', 'chiven', 'chivan', 'xiven', 'guiant',
                'guiar', 'guien', 'escrit', 'escrites', 'signes', 'paraules', 'lletres', 'palabras', 'anecdotas', 'textos',
                'escrito', 'anotacions', 'rotulador', 'escritos', 'textes', 'missagtes', 'pistes', 'habladora', 'dibujado',
                'marcas', 'dice', 'decía', 'decia', 'notaciones', 'paso'],
               ['sube', 'subía', 'subia', 'subir', 'raise', 'raised', 'puja', 'surt', 'pujava', 'sortia', 'salia', 'demás',
                'demés', 'gritar', 'cridar', 'gritaba', 'cariño', 'carinyu', 'amor', 'love', 'ascendió', 'ascendía', 'ascender',
               'eleva', 'pujant', 'sale', 'sola', 'pujaven', 'llamar', 'subida', 'pujar', 'emergeix', 'suba', 'ascensor',
               'elevar'],
                ['as','ace', 'dos', 'two', 'tres', 'three', 'cuatro', 'quatre', 'four', 'cinc', 'cinco', 'five', 'seis', 
              'sis', 'six', 'siete', 'set', 'seven', 'ocho', 'vuit', 'eight', 'nueve', 'nou', 'nine', 'diez',
              'deu', 'ten', 'jota', 'jotas', 'jack', 'dama', 'damas', 'reina', 'reinas', 'queen', 'rey', 'rei', 'reis', 'reyes', 
               'king', 'picas', 'piques', 'spades', 'treboles', 'tréboles', 'trebol', 'trèbol', 'trèbols', 'trebols', 'culbs',
              'diamantes', 'rombos', 'diamants', 'rombes', 'diamonts', 'diamonds', 'corazones', 'corazón', 'cors', 'cor', 'heart',
              'hearts'],
               ['chico', 'chica', 'raquel', 'espectador', 'espectadora']]


#8- Doncs recordo el joc de cartes amb frases escrites, que crec que va ser el joc final. 

###                
Saliencias_words={}
Saliencias_words['1']=saliencias_1
Saliencias_words['2']=saliencias_2
Saliencias_words['3']=saliencias_3
Saliencias_words['4']=saliencias_4
Saliencias_words['5']=saliencias_5
Saliencias_words['6']=saliencias_6
Saliencias_words['7']=saliencias_7
Saliencias_words['8']=saliencias_8


###
Saliencias_titles={}
Saliencias_titles['1'] = ['joker', 'estuche', 'montones', 'color', 'encore', 'carta1', 'persona1']    
Saliencias_titles['2'] = ['frotar', 'dinerito', 'carta2', 'persona2', 'oportunidad', 'desaparecer', 'oreja']  
Saliencias_titles['3'] = ['tapadas', 'viajes']  
Saliencias_titles['4'] = ['alma', 'predicción', 'carta4', 'persona4'] 
Saliencias_titles['5'] = ['material', 'vaso', 'mesa', 'manos', 'salian'] 
Saliencias_titles['6'] = ['deletreo', 'tres', 'española', 'cualquiera', 'fallido', 'persona6'] 
Saliencias_titles['7'] = ['blancas', 'granada', 'tiempo', 'reunión', 'carta7'] 
Saliencias_titles['8'] = ['instrucciones', 'sube', 'carta8', 'persona8'] 






def extract_saliencias(text, efecto_): #### le doy el texto completo y el numero de efecto del que quiero la saliencia
    #### divide el texto en frases y agrupa estas frases en efectos
    #### luego analiza las frases del efecto que le hemos dicho (las del diccionario con '2' y busca saliencias)
    efectos, list_words = effects_detected(text)
    sentences, dict_sentences = separate_sentences_by_effect(text) ## frases agrupadas por efecto
    #######
    saliencias_txt = []  
    if efecto_ not in ['22', '23', '0', '9']: ## most of the cases
        eff_sentences = dict_sentences.get(efecto_)   
        if eff_sentences:
            for sent_ in dict_sentences[efecto_]: ## each sentence by separate
                for idx, salience_list in enumerate(Saliencias_words[efecto_]): 
                    ## permite que una misma saliencia se recuerde mas de una vez si esta en distintas frases del mismo efecto.
                    ## salience_list es una lista con todas las palabras usadas para buscar una slaiencia específica.
                    ## se buscan todas las saliencias del efecto en cada frase (solo las de ese efecto!)
                    if any(word in sent_.split(' ') for word in salience_list): 
                        #solo que haya 1 plabra de la lista en esa frase ya esta la saliencia (si hay 5 de ellas, solo se guarda 1)
                        saliencias_txt.append(Saliencias_titles[efecto_][idx])
                ####
                todas_palabras = list(itertools.chain.from_iterable(Saliencias_words[efecto_]))
                if any(word in sent_.split(' ') for word in todas_palabras) == False:
                    ## si en una frase no hay ninguna palabra de ninguna slaiencia, guarda 'otro'
                    ## problema: te puedes dejar saliencias (mal escritas) si ya ha encontrado alguna otra slaiencia en la misma frase
                    saliencias_txt.append('otro') 
        ##
        ### unique() en saliencias_txt?? o permitir que haya una saliencia mas de una vez?? 
        ### De momento, se permiten repetidas sólo si están en frases distintas ('división memoria')  
        else: ## if efecto_ is not in the dictionary of sentences (eff_sentences is nothing)
            saliencias_txt=[]
    
    
    if efecto_ in ['22', '23', '0', '9']:
        if efecto_ in efectos:
            if efecto_ == '22':
                saliencias_txt.append('monedón')
            elif efecto_ == '23':
                saliencias_txt.append('técnico')
            elif efecto_ == '0':
                saliencias_txt.append('m_interrogante')
            elif efecto_ == '9':
                saliencias_txt.append('c_interrogante')
            
    
        
    ##        
    return saliencias_txt



In [25]:
## Crear matriz de análisis (yendo fila a fila en el excel)
standard_ = [1,2,2,2,3,4,5,6,7,8,0,9]
reversed_ = [8,7,6,5,4,3,2,2,2,1,0,9]
orders={'standard':standard_, 'reversed':reversed_}

all_subjs=[]



for s in range(len(df_data)): ### hayq ue poner las tres hojas en una sola, una debajo de otra
    #print(s)
    times_with_resp = [type(df_data.iloc[s][['justo después', '10 días después', '1 mes y medio después', '4 meses y medio después']][x])==str for x in range(4)]
    n_times_reported = len(times_with_resp) 
    if n_times_reported>1:
        subj_frames=[]
    
    ### Si no contestan, no está en el análisis (todo son 1 si recuerdo o 0 si no recuerdo)
    if times_with_resp==[True, False, False, False]:
        times_rep = [1]
    if times_with_resp==[True, True, False, False]:
        times_rep = [1,2]
    if times_with_resp==[True, False, True, False]:
        times_rep = [1,3]
    if times_with_resp==[True, False, False, True]:
        times_rep = [1,4]
    if times_with_resp==[False, True, False, True]:
        times_rep = [2,4] 
    if times_with_resp==[False, False, True, True]:
        times_rep = [3,4]            
    if times_with_resp==[True, True, True, False]:
        times_rep = [1,2,3]
    if times_with_resp==[True, True, False, True]:
        times_rep = [1,2, 4]
    if times_with_resp==[True, False, True, True]:
        times_rep = [1,3, 4]
    if times_with_resp==[True, True, True, True]:
        times_rep = [1, 2, 3, 4]
        
    for t in times_rep:
        data_t = np.zeros((12,6))
        data_t[:,1] = [1,2,22,23,3,4,5,6,7,8,0,9] # lista de juegos
        #data_t[:,2] = df_data.iloc[s].asiento # asiento
        horario = df_data.iloc[s]['session']
        #
        if horario==16:
            sess=1
            order='standard'
        elif horario==17:
            sess=2
            order='reversed'
        elif horario==19:
            sess=3
            order='standard'
        data_t[:,2] = sess #horario
        data_t[:,3] = s ### subject number
        data_t[:,4] = orders[order]
        data_t[:,5] = t
        ##### Recuerdo y Saliencias
        if t==1:
            col_saliencias = 'justo después'
        elif t==2:
            col_saliencias ='10 días después'
        elif t==3:
            col_saliencias = '1 mes y medio después'
        elif t==4:
            col_saliencias = '4 meses y medio después'
        ##
        text = df_data.iloc[s][col_saliencias]
        efectos = effects_detected(text)[0]
        all_efectos = np.array(['1', '2', '22', '23', '3', '4', '5', '6', '7', '8', '0', '9'])
        recuerdo = np.isin(all_efectos, efectos)*1
        data_t[:,0]=recuerdo 
        saliencias = [extract_saliencias(text, eff) for eff in all_efectos]
       

        data_t=pd.DataFrame(data_t)
        data_t.columns=['recuerdo', 'juego', 'session', 'subj', 'orden', 'tiempo']    
        data_t['saliencias'] = saliencias
        #
        #data_t['nombre'] = df_data.iloc[s]['nombre y apellidos']
        if n_times_reported>1:
            subj_frames.append(data_t)




    if n_times_reported>1:
        subj = pd.concat(subj_frames)
    else:
        subj = data_t


    all_subjs.append(subj)



df_ = pd.concat(all_subjs, ignore_index=True)


In [26]:
## asignacion de numero de saliencias por detección
df_['n_saliencias'] = [len(df_['saliencias'].iloc[i]) for i in range(0, len(df_))]

In [27]:
#### Changes
df_ = df_.loc[(df_['subj']!=100)]
df_.reset_index();

idx = df_.loc[(df_['subj']==115) & (df_['session']==3) & (df_['tiempo']==2) & (df_['juego']==1)].index[0]
df_.set_value(idx, 'saliencias' , ['joker']);

idx = df_.loc[(df_['subj']==26) & (df_['session']==1) & (df_['tiempo']==1) & (df_['juego']==6)].index[0]
df_.set_value(idx, 'saliencias' , ['carta6']);

idx = df_.loc[(df_['subj']==26) & (df_['session']==1) & (df_['tiempo']==2) & (df_['juego']==6)].index[0]
df_.set_value(idx, 'saliencias' , ['carta6', 'española', 'persona6']);

idx = df_.loc[(df_['subj']==26) & (df_['session']==1) & (df_['tiempo']==3) & (df_['juego']==6)].index[0]
df_.set_value(idx, 'saliencias' , ['carta6', 'española']);

idx = df_.loc[(df_['subj']==112) & (df_['session']==3) & (df_['tiempo']==1) & (df_['juego']==1)].index[0]
df_.set_value(idx, 'saliencias' , ['joker']);

idx = df_.loc[(df_['subj']==36) & (df_['session']==1) & (df_['tiempo']==1) & (df_['juego']==1)].index[0]
df_.set_value(idx, 'saliencias' , ['joker']);

idx = df_.loc[(df_['subj']==36) & (df_['session']==1) & (df_['tiempo']==2) & (df_['juego']==1)].index[0]
df_.set_value(idx, 'saliencias' , ['joker', 'color']);

idx = df_.loc[(df_['subj']==36) & (df_['session']==1) & (df_['tiempo']==3) & (df_['juego']==1)].index[0]
df_.set_value(idx, 'saliencias' , ['joker', 'color']);

idx = df_.loc[(df_['subj']==36) & (df_['session']==1) & (df_['tiempo']==4) & (df_['juego']==1)].index[0]
df_.set_value(idx, 'saliencias' , ['joker']);

idx = df_.loc[(df_['subj']==45) & (df_['session']==2) & (df_['tiempo']==2) & (df_['juego']==6)].index[0]
df_.set_value(idx, 'saliencias' , ['carta6']);

idx = df_.loc[(df_['subj']==45) & (df_['session']==2) & (df_['tiempo']==3) & (df_['juego']==6)].index[0]
df_.set_value(idx, 'saliencias' , ['carta6', 'persona6']);

idx = df_.loc[(df_['subj']==61) & (df_['session']==2) & (df_['tiempo']==1) & (df_['juego']==1)].index[0]
df_.set_value(idx, 'saliencias' , ['color', 'joker']);

idx =  df_.loc[(df_['subj']==84) & (df_['session']==3) & (df_['tiempo']==1) & (df_['juego']==1)].index[0]
df_.set_value(idx, 'saliencias' , ['color', 'montones', 'encore']);

idx = df_.loc[(df_['subj']==31) & (df_['session']==1) & (df_['tiempo']==2) & (df_['juego']==1)].index[0]
df_.set_value(idx, 'saliencias' , ['joker']);

idx = df_.loc[(df_['subj']==31) & (df_['session']==1) & (df_['tiempo']==3) & (df_['juego']==1)].index[0]
df_.set_value(idx, 'saliencias' , ['joker']);



C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':
C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':
C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py

In [29]:
df_.to_excel('df_proces.xlsx')